# The McDonald's Diet

In this exercise, we will construct a variety of daily meal plans made up entirely of items from McDonald's. Let's to see what data I have.

## Problem 1

You first task is to read in McDonaldsmenu.csv data set and return a dataframe with  only the "Item" and "Calories" columns, as well as any column whose column name contains a "%".

**IMPORTANT: When you read in the data, you must use the relative file path, i.e. the path Data/McDonaldsmenu.csv.**


In [1]:
import pandas as pd
from gurobipy import *

In [2]:
def problem1():

    import numpy as np
    df_mcdonalds = pd.read_csv("Data/McDonaldsmenu.csv")
    nutrition = []
    for col_name in df_mcdonalds.columns:
        if "(% Daily Value)" in col_name:
            nutrition.append(col_name)
    nutrition.append("Calories")
    nutrition.append("Item")
    df_nutrition = df_mcdonalds[nutrition]
    return df_nutrition

In [3]:
problem1()

,Total Fat (% Daily Value),Saturated Fat (% Daily Value),Cholesterol (% Daily Value),Sodium (% Daily Value),Carbohydrates (% Daily Value),Dietary Fiber (% Daily Value),Vitamin A (% Daily Value),Vitamin C (% Daily Value),Calcium (% Daily Value),Iron (% Daily Value),Calories,Item
0,20,25,87,31,10,17,10,0,25,15,300,Egg McMuffin
1,12,15,8,32,10,17,6,0,25,8,250,Egg White Delight
2,35,42,15,33,10,17,8,0,25,10,370,Sausage McMuffin
3,43,52,95,36,10,17,15,0,30,15,450,Sausage McMuffin with Egg
4,35,42,16,37,10,17,6,0,25,10,400,Sausage McMuffin with Egg Whites
...,...,...,...,...,...,...,...,...,...,...,...,...
255,26,44,14,12,27,4,15,0,40,8,510,McFlurry with Oreo Cookies (Small)
256,35,58,19,16,35,5,20,0,50,10,690,McFlurry with Oreo Cookies (Medium)
257,17,29,9,8,18,2,10,0,25,6,340,McFlurry with Oreo Cookies (Snack)
258,50,76,20,17,38,9,20,0,60,6,810,McFlurry with Reese's Peanut Butter Cups (Medium)


In [4]:
df = problem1()
assert df.shape == (260,12)

## Problem 2

 Let's create daily meal plan from these items that have to abide by the following constraints:

* Any particular item can only be ordered at most twice
* We have to satisfy the daily % values:
    - $\leq$ constraints for: Total Fat, Saturated Fat, Cholesterol, Sodium, Carbohydrates
    - $\geq$ constraint for the others
   
 

We will solve the problem with the following objective:

* Minimize calories

Return the number of calories in the optimal solution as well as a dictionary that has a key for each item ordered in the optimal solution, whose value is the number of times that item should be ordered.

**Hint: You should be calling your function from problem 1 to read in the data**

In [5]:
def problem2():
    
    df_nutrition = problem1()
    total_cals = None
    m = Model()
    order_dictionary = {}
    menu = {}
    length = 0 
    for index, row in df_nutrition.iterrows():
        item_name = df_nutrition.loc[index, 'Item']
        menu[item_name] = m.addVar(lb = 0, ub = 2, vtype = GRB.INTEGER, name = "x_%s" % item_name)
    m.setObjective(expr = quicksum(menu[item_name] * df_nutrition.loc[index, "Calories"]\
                                   for index, item_name in df_nutrition['Item'].iteritems()), sense = GRB.MINIMIZE)

    nutrients = ["Total Fat (% Daily Value)", "Saturated Fat (% Daily Value)", "Cholesterol (% Daily Value)", "Sodium (% Daily Value)", "Carbohydrates (% Daily Value)", \
                "Dietary Fiber (% Daily Value)", "Calcium (% Daily Value)", "Iron (% Daily Value)", "Vitamin C (% Daily Value)", "Vitamin A (% Daily Value)"]
    for nutrient in nutrients:
        if nutrient in df_nutrition.columns:
            if nutrient in ["Total Fat (% Daily Value)", "Saturated Fat (% Daily Value)", "Cholesterol (% Daily Value)", "Sodium (% Daily Value)", "Carbohydrates (% Daily Value)"]:
                m.addConstr(quicksum(menu[item_name] * df_nutrition.loc[index, nutrient] for index, item_name in df_nutrition['Item'].iteritems()) <= 100)
            else:
                m.addConstr(quicksum(menu[item_name] * df_nutrition.loc[index, nutrient] for index, item_name in df_nutrition['Item'].iteritems()) >= 100)
                            
    m.setParam( 'OutputFlag', False )
    m.optimize()
    
    total_cals = m.objVal
    
    for v in m.getVars():
        if v.X > 0:
            item_name = v.varName.split("_")[1]
            order_dictionary[item_name] = int(v.X)
    

    return total_cals, order_dictionary



In [6]:
problem2()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-12


(1640.0,
 {'Fruit & Maple Oatmeal without Brown Sugar': 1,
  'Hamburger': 2,
  'Premium Southwest Salad (without Chicken)': 2,
  'Side Salad': 2,
  'Chocolate Chip Cookie': 2,
  'Fat Free Chocolate Milk Jug': 2})

In [7]:
total_cals, order_dictionary = problem2()
assert  total_cals == 1640

In [8]:
total_cals, order_dictionary = problem2()
assert len(order_dictionary.keys()) == 6
assert order_dictionary['Premium Southwest Salad (without Chicken)'] == 2